In [2]:
# ==== System info ====
!nvidia-smi -L || echo "CPU-only runtime"
!python --version

# ==== Install AutoGluon and Kaggle ====
!pip install -U pip
!pip install -U "autogluon>=1.0" kaggle

# ==== Ensure folders exist ====
import os
for d in ["data","artifacts"]:
    os.makedirs(d, exist_ok=True)
print("Setup complete. Remember: Edit ▸ Notebook settings ▸ uncheck 'Omit code cell output when saving'.")


GPU 0: Tesla T4 (UUID: GPU-42c4c1a8-960a-1b90-ab3a-61fce47e18ae)
Python 3.12.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install -U --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 129.5 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 143.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 36.2 MB/s  0:00:09
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 60.6 MB/s  0:00:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 49.9 MB/s  0:00:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 63.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 66.3 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 71.5 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 70.7 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 144.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 155.9 MB/

In [4]:
!pip install autogluon --quiet


In [1]:
!nvidia-smi


Tue Oct 14 08:10:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# ==== Kaggle API setup ====
from google.colab import files
from pathlib import Path
import os

print("Upload kaggle.json (Kaggle ▸ Account ▸ Create New API Token)")
uploaded = files.upload()  # choose kaggle.json from your computer

Path("/root/.kaggle").mkdir(parents=True, exist_ok=True)
with open("/root/.kaggle/kaggle.json","wb") as f:
    f.write(uploaded['kaggle.json'])
os.chmod("/root/.kaggle/kaggle.json", 0o600)

!kaggle competitions list -s fraud | head -n 5


In [ ]:
# ---- Download & unzip ----
!kaggle competitions download -c ieee-fraud-detection -p data
!unzip -qo data/ieee-fraud-detection.zip -d data/ieee
print("Files in data/ieee:", !ls -1 data/ieee | head -n 10)


In [ ]:
# ---- Load & merge ----
import pandas as pd
train_tr = pd.read_csv('data/ieee/train_transaction.csv')
train_id = pd.read_csv('data/ieee/train_identity.csv')
test_tr  = pd.read_csv('data/ieee/test_transaction.csv')
test_id  = pd.read_csv('data/ieee/test_identity.csv')

train = train_tr.merge(train_id, how='left', on='TransactionID')
test  = test_tr.merge(test_id,  how='left', on='TransactionID')

print("Shapes:", train.shape, test.shape)
print("Target distribution:", train['isFraud'].value_counts(normalize=True))


In [ ]:
# ---- Optional speed tweak: downcast floats ----
for df in (train, test):
    for c in df.select_dtypes(include=['float64']).columns:
        df[c] = df[c].astype('float32')
print('Downcast done')


In [ ]:
# ---- Train AutoGluon ----
from autogluon.tabular import TabularPredictor

label = 'isFraud'
predictor = TabularPredictor(label=label, problem_type='binary', path='ag_ieee/').fit(
    train_data=train,
    time_limit=1800,                  # adjust for longer/better training (e.g., 3600)
    presets='medium_quality_faster_train',  # swap to 'best_quality' if you have more time
)
print("Training complete.")


In [ ]:
# ---- Leaderboard & Feature Importance → artifacts/ ----
lb = predictor.leaderboard(silent=True)
fi = predictor.feature_importance(train)

lb_path = 'artifacts/ieee_fraud_leaderboard.csv'
fi_path = 'artifacts/ieee_fraud_feature_importance.csv'
lb.to_csv(lb_path, index=False)
fi.to_csv(fi_path)

print("Saved:", lb_path, fi_path)
lb.head(10), fi.head(10)


In [ ]:
# ---- Predictions for Kaggle submission ----
probas = predictor.predict_proba(test)
sub = pd.DataFrame({
    'TransactionID': test['TransactionID'],
    'isFraud': probas[1],  # probability of positive class
})
sub_path = 'artifacts/ieee_fraud_submission.csv'
sub.to_csv(sub_path, index=False)
print("Saved:", sub_path)
sub.head()
